<a href="https://colab.research.google.com/github/vi-mcw196/IBM-coursera/blob/main/Reddit_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
 # !pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import praw
import plotly.express as px

In [4]:
client_id = 'TaYNLleb7mylS1UMSattXA'
client_secret = 'Kr7LqCpOB6qgsr8AHQ5iZ50xDOcr_w'
user_agent = 'PWR project Data Science 1.0 by /u/VIktor Moldovan 260311'

subreddits = ['CryptoCurrency', 'Jokes', 'anime']

In [ ]:
def collect_usernames(reddit, subreddit_name):
    subreddit = reddit.subreddit(subreddit_name)
    usernames = set()
    for post in subreddit.hot(limit=100):
        usernames.add(str(post.author))
        post.comments.replace_more(limit=None)
        for comment in post.comments.list():
            usernames.add(str(comment.author))
            if len(usernames) >= 100:
                break
        if len(usernames) >= 100:
            break
    return list(usernames)

In [ ]:
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
topic_to_user_dict = dict()
for subreddit in subreddits:
        users = collect_usernames(reddit, subreddit)
        topic_to_user_dict[subreddit] = users

In [ ]:
topic_to_user_dict

In [ ]:
def get_user_activity(reddit, subreddit_name, user_list):
    subreddit = reddit.subreddit(subreddit_name)
    posts_data = []
    comments_data = []

    for post in subreddit.hot(limit=100):
        if str(post.author) in user_list:
            posts_data.append([post.title, str(post.author), post.score, post.id, post.selftext, post.created])

        post.comments.replace_more(limit=None)
        for comment in post.comments.list():
            if str(comment.author) in user_list:
                comments_data.append([post.id, str(comment.author), comment.body, comment.score, comment.created])

    posts_df = pd.DataFrame(posts_data, columns=['Title', 'Author', 'Score', 'Post_ID', 'Body', 'Created'])
    comments_df = pd.DataFrame(comments_data, columns=['Post_ID', 'Author', 'Comment', 'Score', 'Created'])

    return posts_df, comments_df


In [ ]:
activity_data = {}

for subreddit in subreddits:
    user_list = topic_to_user_dict.get(subreddit, [])
    posts_df, comments_df = get_user_activity(reddit, subreddit, user_list)
    activity_data[subreddit] = {'posts': posts_df, 'comments': comments_df}

In [ ]:
for subreddit in subreddits:
     # Save posts and comments DataFrames to CSV files
    posts_df.to_csv(f'{subreddit}_posts.csv', index=False)
    comments_df.to_csv(f'{subreddit}_comments.csv', index=False)

In [ ]:
import praw
import pandas as pd

client_id = 'TaYNLleb7mylS1UMSattXA'
client_secret = 'Kr7LqCpOB6qgsr8AHQ5iZ50xDOcr_w'
user_agent = 'PWR project Data Science 1.0 by /u/VIktor Moldovan 260311'
subreddits = ['Jokes', 'anime']

def collect_user_activity(reddit, subreddit_name, limit=120, username_limit=3000):
    subreddit = reddit.subreddit(subreddit_name)
    usernames = set()
    posts_data = []
    comments_data = []

    for post in subreddit.hot(limit=limit):
        author = str(post.author)
        usernames.add(author)
        posts_data.append([post.title, author, post.score, post.id, post.selftext, post.created])

        post.comments.replace_more(limit=None)
        for comment in post.comments.list():
            comment_author = str(comment.author)
            usernames.add(comment_author)
            if comment_author in usernames:
                comments_data.append([post.id, comment_author, comment.body, comment.score, comment.created])

            if len(usernames) >= username_limit:
                break
        if len(usernames) >= username_limit:
            break

    posts_df = pd.DataFrame(posts_data, columns=['Title', 'Author', 'Score', 'Post_ID', 'Body', 'Created'])
    comments_df = pd.DataFrame(comments_data, columns=['Post_ID', 'Author', 'Comment', 'Score', 'Created'])

    return usernames, posts_df, comments_df


def main():
    reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
    activity_data = {}

    for subreddit in subreddits:
        usernames, posts_df, comments_df = collect_user_activity(reddit, subreddit)
        activity_data[subreddit] = {'users': list(usernames), 'posts': posts_df, 'comments': comments_df}

        posts_df.to_csv(f'{subreddit}_posts.csv', index=False)
        comments_df.to_csv(f'{subreddit}_comments.csv', index=False)


if __name__ == '__main__':
    main()

In [ ]:
subreddits = ['CryptoCurrency', 'Jokes', 'anime']
posts_data = pd.DataFrame()
comments_data = pd.DataFrame()

for subreddit in subreddits:
    posts_df = pd.read_csv(f'{subreddit}_posts.csv')
    comments_df = pd.read_csv(f'{subreddit}_comments.csv')

    posts_df['Subreddit'] = subreddit
    comments_df['Subreddit'] = subreddit

    posts_data = posts_data.append(posts_df, ignore_index=True)
    comments_data = comments_data.append(comments_df, ignore_index=True)

In [22]:
posts_data['Created'] = pd.to_datetime(posts_data['Created'], unit='s')
comments_data['Created'] = pd.to_datetime(comments_data['Created'], unit='s')

In [27]:
posts_data.dtypes

Title                object
Author               object
Score                 int64
Post_ID              object
Body                 object
Created      datetime64[ns]
Subreddit            object
dtype: object

In [28]:
comments_data.dtypes

Post_ID              object
Author               object
Comment              object
Score                 int64
Created      datetime64[ns]
Subreddit            object
dtype: object

In [16]:
print(posts_data.head())
print(posts_data['Subreddit'].value_counts())

                                               Title             Author  \
0          Moons Update - The Return of Moons Flairs   CryptoMaximalist   
1  Megathread: Binance CEO CZ Steps Down As Part ...         CryptoMods   
2  I’ve read the 90 page complaint lodged against...  Open_Platypus1573   
3  Binance CEO CZ In Discussions To Step Down As ...           GabeSter   
4  Tether deposited $1bn at investment group foun...        Turtlecomuk   

   Score  Post_ID                                               Body  \
0     33  180koxz  #The Return of [Moons](https://i.imgur.com/I0s...   
1      4  180mp04  This story is developing and may be related to...   
2    387  180dnxe  I’ve read the SEC’s 90 page official complaint...   
3     26  180lnye                                                NaN   
4     42  180go2i                                                NaN   

              Created       Subreddit  
0 2023-11-21 16:17:08  CryptoCurrency  
1 2023-11-21 17:42:35  CryptoCurrenc

In [25]:
# Aggregate data
aggregated_data = posts_data.groupby([posts_data['Created'].dt.date, 'Subreddit']).size().reset_index(name='Post_Count')
aggregated_comments = comments_data.groupby([comments_data['Created'].dt.date, 'Subreddit']).size().reset_index(name='Comment_Count')

# Plotting
fig = px.line(aggregated_data, x='Created', y='Post_Count', color='Subreddit', labels={'Created': 'Date', 'Post_Count': 'Number of Posts'})
fig.update_layout(title='Number of Posts Over Time for Each Subreddit')
fig.show()

fig_comments = px.line(aggregated_comments, x='Created', y='Comment_Count', color='Subreddit', labels={'Created': 'Date', 'Comment_Count': 'Number of Comments'})
fig_comments.update_layout(title='Number of Comments Over Time for Each Subreddit')
fig_comments.show()

In [26]:
# Aggregate across all topics
total_posts_per_day = posts_data.groupby(posts_data['Created'].dt.date).size().reset_index(name='Total_Posts')
total_comments_per_day = comments_data.groupby(comments_data['Created'].dt.date).size().reset_index(name='Total_Comments')

# Plotting aggregated data
fig_total = px.line(total_posts_per_day, x='Created', y='Total_Posts', labels={'Created': 'Date', 'Total_Posts': 'Total Number of Posts'})
fig_total.update_layout(title='Total Number of Posts Across All Topics Over Time')
fig_total.show()

fig_total_comments = px.line(total_comments_per_day, x='Created', y='Total_Comments', labels={'Created': 'Date', 'Total_Comments': 'Total Number of Comments'})
fig_total_comments.update_layout(title='Total Number of Comments Across All Topics Over Time')
fig_total_comments.show()


##Obliczanie metryk

###Dla postów:
- Średni wynik na post: Średnia z kolumny "Score" dla każdego użytkownika.
- Liczba komentarzy na post: Wymaga to połączenia z danymi komentarzy w celu zliczenia komentarzy na post.
- Częstotliwość postów: Liczba postów na użytkownika w czasie.

###Dla komentarzy:
- Średni wynik komentarzy: Średnia kolumny "Score" dla komentarzy dla każdego użytkownika.

In [45]:
# Select a random sample of users
sample_size = 100
random_users = posts_data['Author'].dropna().sample(sample_size).tolist()


In [46]:
def fetch_top_posts_by_user(reddit, username, limit=100):
    user = reddit.redditor(username)
    posts = []

    try:
        for post in user.submissions.top(limit=limit):
            posts.append([post.title, post.score, post.id, post.selftext, post.created])
    except Exception as e:
        print(f"Error fetching data for user {username}: {e}")

    return pd.DataFrame(posts, columns=['Title', 'Score', 'Post_ID', 'Body', 'Created'])


In [ ]:
for user in random_users:
    reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
    user_posts = fetch_top_posts_by_user(reddit, user)
    posts_data = posts_data.append(user_posts, ignore_index=True)

In [ ]:
random_users

In [50]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, subplot_titles=('Avg Post Score', 'Comments per Post', 'Avg Comment Score', 'Post Frequency'))

for user in random_users:
    user_posts = posts_data[posts_data['Author'] == user].groupby(pd.Grouper(key='Created', freq='2D')).agg({
        'Score': 'mean',
        'Post_ID': 'count'
    })

    user_comments = comments_data[comments_data['Author'] == user].groupby(pd.Grouper(key='Created', freq='2D')).agg({
        'Score': 'mean'
    })

    combined_data = pd.merge(user_posts, user_comments, how='left', on='Created')
    combined_data['Comments_per_Post'] = combined_data['Post_ID'] / combined_data['Score_y'].fillna(1)  # Calculate comments per post

    fig.add_trace(go.Scatter(x=user_posts.index, y=user_posts['Score'], mode='lines', name=f'Avg Post Score - {user}'), row=1, col=1)
    fig.add_trace(go.Scatter(x=combined_data.index, y=combined_data['Comments_per_Post'], mode='lines', name=f'Comments/Post - {user}'), row=2, col=1)
    fig.add_trace(go.Scatter(x=user_comments.index, y=user_comments['Score'], mode='lines', name=f'Avg Comment Score - {user}'), row=3, col=1)
    fig.add_trace(go.Scatter(x=user_posts.index, y=user_posts['Post_ID'], mode='lines', name=f'Post Frequency - {user}'), row=4, col=1)

fig.update_layout(height=1200, title_text="User Popularity Analysis Over Time")
fig.show()
